In [17]:
import re

import pandas as pd
from bs4 import BeautifulSoup

In [18]:
with open("pid_table.html", "r") as f:
    soup = BeautifulSoup(f.read(), "html.parser")
table = soup.find('table')

In [19]:
table_header = table.find('thead')
col_headers = [el.text.strip() for el in table_header.find_all("th")]
col_headers[col_headers.index("Data bytes\n                    returned")] = "Data bytes returned"

In [20]:
data = []
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [23]:
df = pd.DataFrame(data, columns=col_headers)
df = df.fillna("")
df = df.rename(
    columns={
        "Formula[a]": "Formula",
        "Data bytes returned": "Data bytes",
        "Min value": "Min",
        "Max value": "Max",
        "PID(Dec)": "PID",
    }
)
df = df.drop("PIDs(hex)", axis=1)
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: " ".join(filter(len, map(str.strip, x.split("\n"))))
    )
df["Formula_raw"] = df["Formula"]
for col in ["Formula"]:
    if col.endswith("_raw"):
        continue
    df[col] = (
        df[col]
        .apply(lambda x: re.sub(r"\\t?frac {([^}]+)}{([^}]+)}", r"(\1/\2)", x))
        .apply(lambda x: re.sub(r"\\displaystyle {([^}]+)}", r"\1", x))
        .apply(lambda x: re.sub(r"{\\displaystyle ([^}]+)}", r"(\1)", x))
        .apply(lambda x: re.sub(r"{|}", r"", x))
    )
df.to_csv("../pid_table.csv", index=False)
df

,PID,Data bytes,Description,Min,Max,Units,Formula,Formula_raw
0,0,4,PIDs supported [$01 - $20],Bit encoded [A7..D0] == [PID $01..PID $20] See...,,,,
1,1,4,Monitor status since DTCs cleared. (Includes m...,Bit encoded. See below,,,,
2,2,2,DTC that caused freeze frame to be stored.,Decoded as in service 3,,,,
3,3,2,Fuel system status,Bit encoded. See below,,,,
4,4,1,Calculated engine load,0,100,%,100 255 A (100/255)A (or A 2.55 (A/2.55) ),100 255 A {\displaystyle {\tfrac {100}{255}}A}...
...,...,...,...,...,...,...,...,...
169,196,8,Exhaust Particulate Control System Diagnostic ...,0,"4,294,967,295",seconds / Count,B5 is Engine Idle RequestB6 is Engine Stop Req...,B5 is Engine Idle RequestB6 is Engine Stop Req...
170,197,4,Fuel Pressure A and B,0,"5,177",kPa,,
171,198,7,Byte 1 - Particulate control - driver induceme...,0,"65,535",h,,
172,199,2,Distance Since Reflash or Module Replacement,0,"65,535",km,,
